In [1]:
# Copyright (c) 2025, InfinityQ Technology Inc.

# Installing Python dependencies
!pip install -r "requirements.txt" --upgrade

import numpy as np
import yfinance as yf
import logging
from utils import *
from titanq import Model, Vtype, Target, S3Storage
import warnings

# Filter out FutureWarning messages due to yfinance using a deprecated pandas keyword
warnings.simplefilter(action='ignore', category=FutureWarning)

## Setting Credentials
The user should configure their TitanQ API key here. For very large problems, the user must also configure an AWS Access key, AWS Secret Access key and AWS Bucket Name.

In [2]:
logging.getLogger('botocore').setLevel(logging.CRITICAL)
logging.getLogger('urllib3').setLevel(logging.CRITICAL)

# Enter your API key here
# Obtain your API key by contacting --> support@infinityq.tech
# Example: TITANQ_DEV_API_KEY = "00000000-0000-0000-0000-000000000000"
TITANQ_DEV_API_KEY = None

# Specify AWS keys and bucket name for solving very large problems
# AWS_ACCESS_KEY = "Access Key"
# AWS_SECRET_ACCESS_KEY = "Secret Access Key"
# AWS_BUCKET_NAME = "Bucket Name"

## Setting Up The Problem
Here we pick an instance from the instances folder and read in the stock symbols (tickers) and their respective weights if given. Then we compute the correlation matrix, and consequently the graph adjacency ($J$) matrix with the chosen value of $\theta$. The user decides the time period and frequency that the correlation matrix is calculated over.

If no weights are given in the instance file, then we use each stock's annualized returns rounded to the nearest integer as their weight. 

The parameter $\theta \in [0, 1]$ is the threshold to determine the level at which two assets are correlated. More specifically, two assets are connected by an edge if the absolute value of the correlation coefficient of their returns is greater than (or equal to) a threshold $\theta$. That is, we have: $\text{asset } i \text{ and } \text{asset } j \text{ are connected } \Leftrightarrow |correlation(r_i, r_j)| \geq \theta$ where $r_i$ and $r_j$ are the respective returns of the assets.



In [ ]:
# Read in tickers and weights from the instance file
instance = "S&P 500"
tickers, weights = read_instance(f"instances/{instance}")

# Fetch historical data from Yahoo Finance
start_date = "2010-01-01"
end_date = "2023-12-31"
period = 'Y'

# Get daily closing prices
close_prices = yf.download(tickers, start=start_date, end=end_date, auto_adjust=True)["Close"][tickers]

# Prune delisted stocks
close_prices = close_prices.dropna(axis=1, how='all')

# Number of stocks in consideration (number of vertices in the graph)
size = len(close_prices.columns)

# Get periodic returns
daily_returns = get_stock_daily_returns(close_prices)
periodic_returns = daily_to_periodic_returns(daily_returns, period)

# Compute correlation matrix and adjacency matrix
corr_matrix = get_stock_corr_matrix(periodic_returns)
theta = 0.2
J_matrix = corr_to_J_matrix(corr_matrix, theta)

# Compute (geometric) mean returns
mean_returns = get_stock_mean_returns(periodic_returns)

# Set weights if no weights were specified in the instance file
if weights is None:
    weights = np.round(np.array(mean_returns, dtype=np.float32))

In [ ]:
# Display results of computations above
print("~~~~~~~~~~~~~ Correlation Matrix ~~~~~~~~~~~~~")
print(corr_matrix)
print()

print("~~~~~~~~~~~~~ Adjacency Matrix ~~~~~~~~~~~~~")
print(J_matrix)
print("Graph density: {:.2%}".format((np.sum(J_matrix)/2)/(size*(size-1)/2)))
print()

print("~~~~~~~~~~~~~ Weights ~~~~~~~~~~~~~")
print(weights)

## Inputting The Problem Into The TitanQ SDK
Here the user should provide the credentials they defined earlier. The user also defines the variables of the problem as well as the type of variable.

The user then injects the weights matrix and the bias vector into the SDK as the objective function to minimize. 

In [5]:
#############
# TitanQ SDK
#############
model = Model(
    api_key=TITANQ_DEV_API_KEY,
    # Insert storage_client parameter and specify corresponding AWS keys and bucket name for solving very large problems
    # storage_client=S3Storage(
    #     access_key=AWS_ACCESS_KEY,
    #     secret_key=AWS_SECRET_ACCESS_KEY,
    #     bucket_name=AWS_BUCKET_NAME
    # )
)

x = model.add_variable_vector('x', size, Vtype.BINARY)

# Hyperparameters to tune
A = 1
B = 0.02

# Setting Objective Using Expression
expr = A*0.5*np.dot(x.T, J_matrix.dot(x))-B*weights.dot(x)
model.set_objective_expression(expr)

# Construct Constraints
mask = np.zeros(size, dtype=np.float32)
mask[0] = 1
constraint_bound = np.array([0, 1], dtype=np.float32)
model.add_inequality_constraint(mask, constraint_bound)

Finally, the user can call the optimize method on the model to solve the problem. The user can specify the maximum runtime of the solver (longer times have a higher probability of finding the optimal solution). They can also tune the other solver hyperparameters which are all defined in the README.

In [6]:
# TitanQ Solver Hyperparameters
coupling_mult = 0.1
timeout_in_secs = 0.1
num_chains = 8
num_engines = 1
Tmin = 0.01
Tmax = 0.64

beta = (1/np.linspace(Tmin, Tmax, num_chains, dtype=np.float32)).tolist()

response = model.optimize(beta = beta, coupling_mult=coupling_mult, timeout_in_secs=timeout_in_secs, num_chains=num_chains, num_engines=num_engines)

## Printing The Results

The list of solutions for each engine can be accessed from the variable label defined before (in this case: ```x```). For each engine we print:
- The solution
- The corresponding stock symbols
- The weight of the solution
- The Ising energy of the solution
- A validation of the solution, i.e. check if the set of nodes is actually independent or not.

Then we print the *best* valid solution (valid solution with the greatest weight). This gives a low-risk portfolio of uncorrelated stocks that is preferrable to the investor.



In [ ]:
# Keep track of the index and the weight of the best solution
best_idx = -1
best_weight = 0

print("-------- ALL ENGINE RESULTS --------")
for idx, solution in enumerate(response.x):
    solution_weight = weights.dot(solution)
    
    # Print all results
    print(f"\n--- Engine {idx + 1} ---")
    print("     x:", solution)
    print("stocks:", [tickers[i] for i in np.nonzero(solution)[0]])
    print("weight:", solution_weight)
    print("energy:", response.computation_metrics().get('solutions_objective_value')[idx])

    # Check if this solution is valid
    violations = verify_independent_set(J_matrix, solution, tickers)
    if violations:
        print("This solution is not valid since the corresponding set of nodes is not independent.")
        print("Try adjusting the hyperparameters further to yield a valid solution.")
        print("The following vertices are connected:")
        for pair in violations:
            print(pair)
    else:
        print("This solution is valid! The corresponding set of nodes is independent.")
        
        # Check if this solution is the best valid one so far
        if best_idx == -1 or solution_weight > best_weight:
            best_idx = idx
            best_weight = solution_weight
    
        
# Print the results of the best valid solution
print("\n-------- BEST VALID SOLUTION --------")

if best_idx == -1:
    print("None of the engines returned valid solutions!")
    print("Try adjusting the hyperparameters further to yield some valid solutions.")
else:
    print(f"--- Engine {best_idx + 1} ---")
    print("     x:", response.x[best_idx])
    print("stocks:", [tickers[i] for i in np.nonzero(response.x[best_idx])[0]])
    print("weight:", best_weight)
    print("energy:", response.computation_metrics().get('solutions_objective_value')[best_idx])